In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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'
    
)

response = llm.invoke("The first person to land on moon was ...")
print (response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2988679/software-development-engineer-data-plane-npd-forwarding-stack-data-plane")
page_data = loader.load().pop().page_content
print(page_data)

Software Development Engineer, Data Plane, NPD Forwarding Stack, Data Plane - Job ID: 2988679 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Software Development Engineer, Data Plane, NPD Forwarding Stack, Data PlaneJob ID: 2988679 | Amazon Data Services, Inc.Apply nowDESCRIPTIONAWS Infrastructure Services owns the design, planning, delivery, and operation of all AWS global infrastructure. In other words, we’re the people who keep the cloud running. We support all AWS data centers and all of the servers, storage, networking, power, and cooling equipment that ensure our customers have continual access to the innovation they rely on. We work on the most challenging problems, with thousands of variables impacting the supply chain — and we’re looking for talented people who want to help.Yo

In [11]:
from langchain_core.prompts import PromptTemplate

# Create the prompt template
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON.
    
    ### VALID JSON (NO PREAMBLE):
    """
)

# Chain the prompt with the model
chain_extract = prompt_extract | llm

# Corrected `invoke` usage
res = chain_extract.invoke({'page_data': page_data})

# Print result content
print(res)


content='```json\n{\n  "role": "Software Development Engineer, Data Plane, NPD Forwarding Stack, Data Plane",\n  "experience": {\n    "years": 3,\n    "type": "non-internship professional software development experience",\n    "design": 2\n  },\n  "skills": [\n    "Linux Kernel",\n    "Rust",\n    "Go",\n    "Java",\n    "C++",\n    "Linux-based solutions",\n    "embedded devices",\n    "networking products"\n  ],\n  "description": "AWS Infrastructure Services owns the design, planning, delivery, and operation of all AWS global infrastructure. We’re looking for talented people who want to help us deliver the highest standards for safety and security while providing seemingly infinite capacity at the lowest possible cost for our customers."\n}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 1411, 'total_tokens': 1578, 'completion_time': 0.607272727, 'prompt_time': 0.099813187, 'queue_time': 0.05641142800000001, 'total_time': 0.707

In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Development Engineer, Data Plane, NPD Forwarding Stack, Data Plane',
 'experience': {'years': 3,
  'type': 'non-internship professional software development experience',
  'design': 2},
 'skills': ['Linux Kernel',
  'Rust',
  'Go',
  'Java',
  'C++',
  'Linux-based solutions',
  'embedded devices',
  'networking products'],
 'description': 'AWS Infrastructure Services owns the design, planning, delivery, and operation of all AWS global infrastructure. We’re looking for talented people who want to help us deliver the highest standards for safety and security while providing seemingly infinite capacity at the lowest possible cost for our customers.'}

In [29]:
import pandas as pd 

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [30]:

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [31]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [32]:
job = json_res
job['skills']

['Linux Kernel',
 'Rust',
 'Go',
 'Java',
 'C++',
 'Linux-based solutions',
 'embedded devices',
 'networking products']

In [33]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ambuj, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Ambuj Shukla, Data Analyst / Tech-focused role student, PSIT Kanpur. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Services for AWS Infrastructure Services

Dear Hiring Manager,

I came across the job description for a Software Development Engineer, Data Plane, NPD Forwarding Stack, Data Plane at AWS Infrastructure Services, and I believe AtliQ, an AI & Software Consulting company, can fulfill your requirements. With our expertise in delivering tailored solutions, we can empower your team to achieve scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team has extensive experience in developing Linux-based solutions, working with embedded devices, and creating networking products. We are well-versed in programming languages such as Linux Kernel, Rust, Go, Java, C++, which aligns with your job requirements. Our expertise in Linux Kernel and Java can be further explored through our portfolio: https://example.com/java-portfolio and https://example.com/linux-based solutions are not available but we have https://example.com/ml-py

In [34]:
job


{'role': 'Software Development Engineer, Data Plane, NPD Forwarding Stack, Data Plane',
 'experience': {'years': 3,
  'type': 'non-internship professional software development experience',
  'design': 2},
 'skills': ['Linux Kernel',
  'Rust',
  'Go',
  'Java',
  'C++',
  'Linux-based solutions',
  'embedded devices',
  'networking products'],
 'description': 'AWS Infrastructure Services owns the design, planning, delivery, and operation of all AWS global infrastructure. We’re looking for talented people who want to help us deliver the highest standards for safety and security while providing seemingly infinite capacity at the lowest possible cost for our customers.'}